# Day 2 - morning

In [28]:
def f(y):
    def g(x):
        z = 10
        return x + y + z
    return g
a = f(3)
a.__closure__[0].cell_contents

3

In [29]:
for i in range(len(a.__closure__)):
    print(a.__closure__[i], len(a.__closure__))

<cell at 0x7f89ae10ffd0: int object at 0x955e80> 1


In [32]:
def counter(f):
    nb_appel = 0
    def wrapper(*args, **dargs):
        nonlocal nb_appel
        nb_appel = nb_appel + 1
        res = f(*args, **dargs)
        print(f"{f.__name__}: {nb_appel} fois")
        return res
        
    return wrapper

@counter
def f():
    pass

f.__name__

'wrapper'

In [34]:
f()

f: 2 fois


In [10]:
a =[1, 2]
def add(x):
    nonlocal a
    a = a + [x]
add(3)
a

SyntaxError: no binding for nonlocal 'a' found (3282031970.py, line 3)

In [11]:
def f():
    pass

def g():
    pass
f = counter(f)
g = counter(g)

In [14]:
f()

f: 3 fois


In [15]:
g()

g: 1 fois


In [21]:
def counter(f):
    def wrapper(*args, **dargs):
        wrapper.nb_appel = wrapper.nb_appel + 1
        res = f(*args, **dargs)
        print(f"{f.__name__}: {wrapper.nb_appel} fois")
        return res
    wrapper.nb_appel = 0 
    return wrapper

@counter
def f():
    pass

f.__name__

'wrapper'

In [24]:
f()

f: 3 fois


In [25]:
f

<function __main__.counter.<locals>.wrapper(*args, **dargs)>

In [37]:
def adder(x):
    def add(y):
        return x + y
    return add
add_5 = adder(5)
add_5(10)

15

In [40]:
class Adder:
    def __init__(self, x):
        self.x = x
    def __call__(self, y):
        return self.x + y
add_5 = Adder(5)
add_5(10)

15

In [41]:
def f(x, y):
    z = 10
    def g():
        nonlocal x, y, z
        x = 100
        y = 200
        z = 300
    return g

In [43]:
class C:
    def __call__(self):
        print('in call')
        
    def __repr__(self):
        return "dans repr"
c = C()
c()
c

in call


dans repr

In [52]:
from functools import wraps

def counter(f):
    #@wraps(f)
    def wrapper(*args, **dargs):
        wrapper.nb_appel = wrapper.nb_appel + 1
        res = f(*args, **dargs)
        print(f"{f.__name__}: {wrapper.nb_appel} fois")
        return res
    wrapper.nb_appel = 0
    wrapper = wraps(f)(wrapper) # équivalent à @
    return wrapper

@counter
def f(a, b=3):
    """
    Calcul de la fonction f
    """
    return a*b

f.__name__, f.__doc__

('f', '\n    Calcul de la fonction f\n    ')

In [53]:
?f

Signature: f(a, b=3)
Docstring: Calcul de la fonction f
File:      /tmp/ipykernel_422/4027679621.py
Type:      function


In [55]:
class NbAppel: # décorateur sous forme de classe
    def __init__(self, f):
        self.appel = 0
        self.f = f
    def __call__(self, *args):
        self.appel = self.appel + 1
        s = f'{self.f.__name__} : {self.appel} appels'
        print(s)
        print(self, args)
        return self.f(*args)

class C:
    @NbAppel # problème : utiliser un décorateur sous forme de fonction
    def ma_methode(self, x):
        self.x = x
        
c = C()
c.ma_methode

In [56]:
from functools import wraps

def counter(f):
    @wraps(f)
    def wrapper(*args, **dargs):
        wrapper.nb_appel = wrapper.nb_appel + 1
        res = f(*args, **dargs)
        print(f"{f.__name__}: {wrapper.nb_appel} fois")
        return res
    wrapper.nb_appel = 0 
    #wrapper = wraps(f)(wrapper)
    return wrapper


class C:
    @counter
    def ma_methode(self, x):
        self.x = x

c = C()
c.ma_methode


<bound method C.ma_methode of <__main__.C object at 0x7f89acbc4b20>>

```
@timer
@nb_calls
def f:
    pass
```
est equivalent à 
```
f=timer(nb_calls(f))
```

## Passer des arguments à un décorateur

In [71]:
import time

def nb_appel(label=''):
    temps_exe_total = 0
    class NbAppel:
        def __init__(self, f):
            self.appel = 0
            self.f = f
        def __call__(self, *args):
            nonlocal temps_exe_total
            self.appel = self.appel + 1
            s = (f'{label} {self.f.__name__} '
                 f': {self.appel} appels')
            print(s)
            t = time.time()
            res = self.f(*args)
            temps_exe_total += time.time() - t
            print("Temps moyen d'exécution : ", temps_exe_total / self.appel)
            return res
    return NbAppel

# equiv f = nb_appels('-->')(f)
# equiv f = NbAppel(f)
@nb_appel("-->") 
def f(a, b):
    sum(range(10_000_000))
    print(a, b)
    
f(1, 2)

--> f : 1 appels
1 2
Temps moyen d'exécution :  0.12647557258605957


In [75]:
f(1, 2)

--> f : 5 appels
1 2
Temps moyen d'exécution :  0.16813154220581056


In [101]:
import time

def nb_appel(label=''):
    temps_exe_total = 0
    def nbappel(f):
        appel = 0
        def wrapper(*args, **dargs):
            nonlocal temps_exe_total
            nonlocal appel
            appel = appel + 1
            s = (f'{label} {f} '
                 f': {appel} appels {label[::-1]}')
            print(s)
            t = time.time()
            res = f(*args)
            temps_exe_total += time.time() - t
            print("Temps moyen d'exécution : ", temps_exe_total / appel)
            return res
        return wrapper
    return nbappel

# equiv f = nb_appels('-->')(f)
# equiv f = nbappel(f)
@nb_appel("---->") 
def f(a, b):
    sum(range(10_000_000))
    print(a, b)
    
f(1, 2)

----> <function f at 0x7f89acb38ca0> : 1 appels >----
1 2
Temps moyen d'exécution :  0.14335250854492188


In [104]:
f(1, 2)

----> <function f at 0x7f89acb38ca0> : 4 appels >----
1 2
Temps moyen d'exécution :  0.14376866817474365


In [81]:
def my_counter(tag): # ICI
    def counter(f):
        nb_appel = 0
        def wrapper(*args, **dargs):
            nonlocal nb_appel
            nb_appel = nb_appel + 1
            res = f(*args, **dargs)
            print(f"{tag} {f.__name__}: {nb_appel} fois") #
            return res

        return wrapper
    return counter #

@my_counter('###')
def f():
    pass

f()

### f: 1 fois


In [83]:
f()

### f: 3 fois


In [105]:
def logfunc(func):
    """
    Décorateur qui log l'activité d'une fonction.
    """
    @wraps(func)
    def wrapper(*args, **kwargs):
        res = func(*args, **kwargs)
        s = f"""
The function *{func.__name__}* was called:
 - positional arguments: {args}
 - named arguments: {kwargs}
 - returned value: {res}
"""
        print(s)
        return res
    return wrapper

def caller_allmethods(cls):
    for name, obj in vars(cls).items():
        if callable(obj):
            setattr(cls, name, logfunc(obj))
    return cls

@caller_allmethods
class C:
    def f(self):
        pass
    def g(self):
        pass
    def h(self):
        pass

#C = caller_allmethods(C)

In [106]:
for name, obj in vars(C).items():
    if callable(obj):
        print(f"{name=} {obj=}")
        setattr(C, name, logfunc(obj))
        #obj = logfunc(obj)

name='f' obj=<function C.f at 0x7f89acb38820>
name='g' obj=<function C.g at 0x7f89acb38dc0>
name='h' obj=<function C.h at 0x7f89acb38d30>


In [107]:
c = C()
c.f(), c.g(), c.h()


The function *f* was called:
 - positional arguments: (<__main__.C object at 0x7f89acc03e80>,)
 - named arguments: {}
 - returned value: None


The function *f* was called:
 - positional arguments: (<__main__.C object at 0x7f89acc03e80>,)
 - named arguments: {}
 - returned value: None


The function *g* was called:
 - positional arguments: (<__main__.C object at 0x7f89acc03e80>,)
 - named arguments: {}
 - returned value: None


The function *g* was called:
 - positional arguments: (<__main__.C object at 0x7f89acc03e80>,)
 - named arguments: {}
 - returned value: None


The function *h* was called:
 - positional arguments: (<__main__.C object at 0x7f89acc03e80>,)
 - named arguments: {}
 - returned value: None


The function *h* was called:
 - positional arguments: (<__main__.C object at 0x7f89acc03e80>,)
 - named arguments: {}
 - returned value: None



(None, None, None)